In [1]:
import quandl  # Access to Sharadar Core US Equities Bundle
api_key = '7B87ndLPJbCDzpNHosH3'

import math
import platform
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np
import torch
import pandas as pd
from IPython.display import display
import time

from datetime import date, datetime, time, timedelta


print("Python version: ", platform.python_version())
print("Pytorch version: {}".format(torch.__version__))

Python version:  3.6.9
Pytorch version: 1.2.0


### Import Label and Ticker_Price Datasets

In [2]:
start_date = '2009-03-01'
end_date = '2011-12-31'
file_name = 'labels_12-31-2011.csv'

y = pd.read_csv("../datasets/sharader/"+file_name)
y = y.sort_values(by=['ticker'])

y = y.set_index('ticker')

y.head()

,appreciation,10bagger,table,permaticker,name,exchange,isdelisted,category,cusips,siccode,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
ticker,,,,,,,,,,,,,,,,,,,,,
A,6.339117,False,SEP,196290,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,3826.0,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
AAAGY,1.275556,False,SEP,120538,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,2834.0,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AACC,1.978723,False,SEP,195323,Asset Acceptance Capital Corp,NASDAQ,Y,Domestic,04543P100,6153.0,...,USD,Michigan; U.S.A,2018-06-13,2015-06-08,2004-02-05,2013-06-13,2002-12-31,2013-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AACG,0.191506,False,SEP,189031,ATA Creativity Global,NASDAQ,N,ADR,00211V106,8200.0,...,USD,China,2020-01-14,2016-02-02,2008-01-29,2020-01-14,2006-03-31,2018-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.atai.net.cn
AACPF,1.041237,False,SEP,190934,Australia Acquisition Corp,NASDAQ,Y,ADR,G06368123 G06368107,6770.0,...,USD,Australia,2018-07-02,2016-02-01,2010-12-20,2012-11-19,2011-06-30,2012-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


In [11]:
tickers = list(y.index)
valid_tickers = pd.Series(tickers, name = 'ticker')

display(valid_tickers.head())
display(len(valid_tickers))

0        A
1    AAAGY
2     AACC
3     AACG
4    AACPF
Name: ticker, dtype: object

6860

In [4]:
prices = pd.read_csv("../datasets/sharader/SHARADAR_SEP.csv")
prices.head()

,ticker,date,open,high,low,close,volume,dividends,closeunadj,lastupdated
0,A,2019-11-29,80.960,81.240,80.470,80.770,835780.0,0.0,80.770,2019-11-29
1,AA,2019-11-29,20.430,20.470,20.158,20.350,1223254.0,0.0,20.350,2019-11-29
2,AACG,2019-11-29,1.813,1.813,1.805,1.805,963.0,0.0,1.805,2019-11-29
3,AAGIY,2019-11-29,40.580,40.580,40.010,40.020,163700.0,0.0,40.020,2019-11-30
4,AAL,2019-11-29,28.920,29.265,28.660,28.740,2970843.0,0.0,28.740,2019-11-29


### Create Matching Input X

In [5]:
X = prices.filter(items=['ticker', 'date', 'close'])
X = X[(prices.date >= start_date) & (prices.date <= end_date)]

In [6]:
X.head()

,ticker,date,close
12740010,A,2011-12-30,34.93
12740011,AACC,2011-12-30,3.91
12740012,AACG,2011-12-30,7.50
12740013,AACPF,2011-12-30,9.86
12740014,AAGIY,2011-12-30,12.53


In [20]:
len(X.columns)

6955

In [7]:
# Pivot the dataframe
X = X.pivot(index ='date', columns ='ticker')
X.columns = X.columns.droplevel() 

X.head()

ticker,A,AAAGY,AACC,AACG,AACPF,AAGIY,AAI,AAIIQ,AAIR,AAL,...,ZRBA,ZSTN,ZUMZ,ZURVY,ZVO,ZVUE,ZXAIY,ZYTO,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2009-03-02,12.68,15.75,3.29,5.18,NaN,NaN,2.54,1.61,NaN,2.47,...,1.55,NaN,7.45,12.75,NaN,0.01,NaN,0.011,1.21,0.84
2009-03-03,12.68,15.75,3.30,5.32,NaN,NaN,2.46,1.61,NaN,2.30,...,1.40,NaN,7.19,12.85,NaN,0.01,NaN,0.011,1.22,0.76
2009-03-04,13.31,16.35,3.33,5.08,NaN,NaN,2.78,1.61,NaN,2.37,...,1.40,NaN,7.17,13.74,NaN,0.01,NaN,0.011,1.22,0.76
2009-03-05,12.54,15.59,3.30,5.08,NaN,NaN,2.56,1.53,NaN,1.97,...,1.40,NaN,6.69,11.91,NaN,0.01,NaN,0.011,1.17,0.58
2009-03-06,12.65,15.97,3.40,5.25,NaN,NaN,2.89,1.80,NaN,2.16,...,1.22,NaN,6.43,11.30,NaN,0.01,NaN,0.011,1.20,0.55


In [10]:
X_valid = X.loc[:, X.columns.isin(valid_tickers)]

X_valid.head()

ticker,A,AAAGY,AACC,AACG,AACPF,AAGIY,AAI,AAIIQ,AAIR,AAL,...,ZRBA,ZSTN,ZUMZ,ZURVY,ZVO,ZVUE,ZXAIY,ZYTO,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2009-03-02,12.68,15.75,3.29,5.18,NaN,NaN,2.54,1.61,NaN,2.47,...,1.55,NaN,7.45,12.75,NaN,0.01,NaN,0.011,1.21,0.84
2009-03-03,12.68,15.75,3.30,5.32,NaN,NaN,2.46,1.61,NaN,2.30,...,1.40,NaN,7.19,12.85,NaN,0.01,NaN,0.011,1.22,0.76
2009-03-04,13.31,16.35,3.33,5.08,NaN,NaN,2.78,1.61,NaN,2.37,...,1.40,NaN,7.17,13.74,NaN,0.01,NaN,0.011,1.22,0.76
2009-03-05,12.54,15.59,3.30,5.08,NaN,NaN,2.56,1.53,NaN,1.97,...,1.40,NaN,6.69,11.91,NaN,0.01,NaN,0.011,1.17,0.58
2009-03-06,12.65,15.97,3.40,5.25,NaN,NaN,2.89,1.80,NaN,2.16,...,1.22,NaN,6.43,11.30,NaN,0.01,NaN,0.011,1.20,0.55


In [12]:
# Save to input file
file_name = '../datasets/sharader/inputs_notfilled_'+end_date+'.csv'
X_valid.to_csv (file_name, header=True)

## Batch Create X_t 

In [22]:
start_date = '2009-03-01'

end_dates = ['2010-12-31',
             '2011-12-31',
             '2012-12-31',
             '2013-12-31',
             '2014-12-31',
             '2015-12-31',
             '2016-12-31',
             '2017-12-31',
             '2018-12-31'
            ]
             
file_names = ['labels_12-31-2010.csv',
              'labels_12-31-2011.csv',
              'labels_12-31-2012.csv',
              'labels_12-31-2013.csv',
              'labels_12-31-2014.csv',
              'labels_12-31-2015.csv',
              'labels_12-31-2016.csv',
              'labels_12-31-2017.csv',
              'labels_12-31-2018.csv'
             ]

for end_date, file_name in zip(end_dates, file_names):
    
    print(end_date, file_name)

    # Generate the valid tickers for the sample period
    y = pd.read_csv("../datasets/sharader/"+file_name)
    y = y.sort_values(by=['ticker'])
    y = y.set_index('ticker')

    tickers = list(y.index)
    valid_tickers = pd.Series(tickers, name = 'ticker')
    print(len(valid_tickers))
    
    # Get closing prices of corresponding tickers
    X = prices.filter(items=['ticker', 'date', 'close'])
    X = X[(prices.date >= start_date) & (prices.date <= end_date)]
    
    # Pivot the dataframe  --> X_t
    X = X.pivot(index ='date', columns ='ticker')
    X.columns = X.columns.droplevel() 

    print(len(X.columns))
    
    X_valid = X.loc[:, X.columns.isin(valid_tickers)]  # Filter dataframe for valid tickers

    # Save X_t to input file 
    out = '../datasets/sharader/inputs_notfilled_'+end_date+'.csv'
    X_valid.to_csv (out, header=True)
    

2010-12-31 labels_12-31-2010.csv
6550
6728
2011-12-31 labels_12-31-2011.csv
6860
6955
2012-12-31 labels_12-31-2012.csv
7115
7276
2013-12-31 labels_12-31-2013.csv
7422
7626
2014-12-31 labels_12-31-2014.csv
7849
8566
2015-12-31 labels_12-31-2015.csv
8703
9024
2016-12-31 labels_12-31-2016.csv
9018
9421
2017-12-31 labels_12-31-2017.csv
9404
9901
2018-12-31 labels_12-31-2018.csv
9881
10501


In [23]:
df = pd.read_csv("../datasets/sharader/inputs_notfilled_2018-12-31.csv")
df = df.set_index('date')

In [24]:
df

,A,AA,AAAGY,AAAP,AAC,AACC,AACG,AACPF,AAGIY,AAI,...,ZUO,ZURVY,ZVO,ZVUE,ZXAIY,ZYME,ZYNE,ZYTO,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2009-03-02,12.68,NaN,15.75,NaN,NaN,3.29,5.180,NaN,NaN,2.54,...,NaN,12.750,NaN,0.01,NaN,NaN,NaN,0.011,1.21,0.84
2009-03-03,12.68,NaN,15.75,NaN,NaN,3.30,5.320,NaN,NaN,2.46,...,NaN,12.850,NaN,0.01,NaN,NaN,NaN,0.011,1.22,0.76
2009-03-04,13.31,NaN,16.35,NaN,NaN,3.33,5.080,NaN,NaN,2.78,...,NaN,13.740,NaN,0.01,NaN,NaN,NaN,0.011,1.22,0.76
2009-03-05,12.54,NaN,15.59,NaN,NaN,3.30,5.080,NaN,NaN,2.56,...,NaN,11.910,NaN,0.01,NaN,NaN,NaN,0.011,1.17,0.58
2009-03-06,12.65,NaN,15.97,NaN,NaN,3.40,5.250,NaN,NaN,2.89,...,NaN,11.300,NaN,0.01,NaN,NaN,NaN,0.011,1.20,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,62.67,25.15,NaN,NaN,1.52,NaN,0.970,NaN,32.335,NaN,...,16.36,28.680,6.73,NaN,0.67,11.08,2.94,NaN,2.69,NaN
2018-12-26,65.54,27.14,NaN,NaN,1.67,NaN,0.998,NaN,32.700,NaN,...,17.81,29.050,6.78,NaN,0.67,11.99,2.92,NaN,2.63,NaN
2018-12-27,66.48,27.16,NaN,NaN,1.49,NaN,0.980,NaN,32.330,NaN,...,17.90,28.769,6.83,NaN,0.62,11.70,2.95,NaN,2.63,NaN


### Backfill and Forward Fill X_t

In [12]:
X_filled = X.fillna(axis=1, method='ffill')
X_filled = X_filled.fillna(0)

In [13]:
X

close                                                         \
date   2009-03-02 2009-03-03 2009-03-04 2009-03-05 2009-03-06 2009-03-09   
ticker                                                                     
A          12.680     12.680     13.310     12.540     12.650     12.440   
AAAGY      15.750     15.750     16.350     15.590     15.970     15.960   
AACC        3.290      3.300      3.330      3.300      3.400      3.400   
AACG        5.180      5.320      5.080      5.080      5.250      5.210   
AACPF         NaN        NaN        NaN        NaN        NaN        NaN   
...           ...        ...        ...        ...        ...        ...   
ZVO           NaN        NaN        NaN        NaN        NaN        NaN   
ZVUE        0.010      0.010      0.010      0.010      0.010      0.010   
ZYTO        0.011      0.011      0.011      0.011      0.011      0.011   
ZYXI        1.210      1.220      1.220      1.170      1.200      1.100   
ZZ          0.840      0.760      0.760      0.580      0.550      0.570   

                                                    ...                        \
date   2009-03-10 2009-03-11 2009-03-12 2009-03-13  ... 2010-12-17 2010-12-20   
ticker                                              ...                         
A          13.430     13.530     13.810     13.990  ...      40.54      40.63   
AAAGY      16.200     16.150     16.500     16.360  ...        NaN        NaN   
AACC        3.870      4.130      5.120      4.850  ...       5.75       5.70   
AACG        4.950      4.810      4.800      4.760  ...       3.67       3.70   
AACPF         NaN        NaN        NaN        NaN  ...        NaN       9.70   
...           ...        ...        ...        ...  ...        ...        ...   
ZVO           NaN        NaN        NaN        NaN  ...      17.75      17.62   
ZVUE        0.010      0.010      0.010      0.010  ...        NaN        NaN   
ZYTO        0.011      0.011      0.012      0.012  ...       0.09       0.11   
ZYXI        1.000      1.000      1.010      1.050  ...       0.60       0.60   
ZZ          0.770      0.810      0.940      0.910  ...       2.88       2.87   

                                                                          \
date   2010-12-21 2010-12-22 2010-12-23 2010-12-27 2010-12-28 2010-12-29   
ticker                                                                     
A           40.83      41.06     41.070      41.92      41.51      41.69   
AAAGY         NaN        NaN        NaN        NaN        NaN        NaN   
AACC         5.75       5.77      5.770       6.04       5.96       6.02   
AACG         3.87       3.81      3.790       3.71       3.71       3.69   
AACPF        9.55       9.50      9.700       9.61       9.61       9.69   
...           ...        ...        ...        ...        ...        ...   
ZVO         18.12      18.41     18.480      18.90      18.82      19.14   
ZVUE          NaN        NaN        NaN        NaN        NaN        NaN   
ZYTO         0.11       0.11      0.105       0.11       0.11       0.11   
ZYXI         0.61       0.62      0.620       0.55       0.55       0.58   
ZZ           2.98       2.95      2.930       2.94       2.95       2.98   

                              
date   2010-12-30 2010-12-31  
ticker                        
A           41.59      41.43  
AAAGY         NaN        NaN  
AACC         5.92       5.93  
AACG         3.65       3.65  
AACPF        9.64       9.65  
...           ...        ...  
ZVO         18.94      19.00  
ZVUE          NaN        NaN  
ZYTO         0.11       0.11  
ZYXI         0.62       0.60  
ZZ           2.92       2.92  

[6728 rows x 465 columns]

In [14]:
X_filled

close                                                         \
date   2009-03-02 2009-03-03 2009-03-04 2009-03-05 2009-03-06 2009-03-09   
ticker                                                                     
A          12.680     12.680     13.310     12.540     12.650     12.440   
AAAGY      15.750     15.750     16.350     15.590     15.970     15.960   
AACC        3.290      3.300      3.330      3.300      3.400      3.400   
AACG        5.180      5.320      5.080      5.080      5.250      5.210   
AACPF       0.000      0.000      0.000      0.000      0.000      0.000   
...           ...        ...        ...        ...        ...        ...   
ZVO         0.000      0.000      0.000      0.000      0.000      0.000   
ZVUE        0.010      0.010      0.010      0.010      0.010      0.010   
ZYTO        0.011      0.011      0.011      0.011      0.011      0.011   
ZYXI        1.210      1.220      1.220      1.170      1.200      1.100   
ZZ          0.840      0.760      0.760      0.580      0.550      0.570   

                                                    ...                        \
date   2009-03-10 2009-03-11 2009-03-12 2009-03-13  ... 2010-12-17 2010-12-20   
ticker                                              ...                         
A          13.430     13.530     13.810     13.990  ...     40.540     40.630   
AAAGY      16.200     16.150     16.500     16.360  ...     20.090     20.090   
AACC        3.870      4.130      5.120      4.850  ...      5.750      5.700   
AACG        4.950      4.810      4.800      4.760  ...      3.670      3.700   
AACPF       0.000      0.000      0.000      0.000  ...      0.000      9.700   
...           ...        ...        ...        ...  ...        ...        ...   
ZVO         0.000      0.000      0.000      0.000  ...     17.750     17.620   
ZVUE        0.010      0.010      0.010      0.010  ...      0.008      0.008   
ZYTO        0.011      0.011      0.012      0.012  ...      0.090      0.110   
ZYXI        1.000      1.000      1.010      1.050  ...      0.600      0.600   
ZZ          0.770      0.810      0.940      0.910  ...      2.880      2.870   

                                                                          \
date   2010-12-21 2010-12-22 2010-12-23 2010-12-27 2010-12-28 2010-12-29   
ticker                                                                     
A          40.830     41.060     41.070     41.920     41.510     41.690   
AAAGY      20.090     20.090     20.090     20.090     20.090     20.090   
AACC        5.750      5.770      5.770      6.040      5.960      6.020   
AACG        3.870      3.810      3.790      3.710      3.710      3.690   
AACPF       9.550      9.500      9.700      9.610      9.610      9.690   
...           ...        ...        ...        ...        ...        ...   
ZVO        18.120     18.410     18.480     18.900     18.820     19.140   
ZVUE        0.008      0.008      0.008      0.008      0.008      0.008   
ZYTO        0.110      0.110      0.105      0.110      0.110      0.110   
ZYXI        0.610      0.620      0.620      0.550      0.550      0.580   
ZZ          2.980      2.950      2.930      2.940      2.950      2.980   

                              
date   2010-12-30 2010-12-31  
ticker                        
A          41.590     41.430  
AAAGY      20.090     20.090  
AACC        5.920      5.930  
AACG        3.650      3.650  
AACPF       9.640      9.650  
...           ...        ...  
ZVO        18.940     19.000  
ZVUE        0.008      0.008  
ZYTO        0.110      0.110  
ZYXI        0.620      0.600  
ZZ          2.920      2.920  

[6728 rows x 465 columns]